In [11]:
import cv2
import numpy as np
import os
import random
from PIL import Image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import joblib  # To save and load trained models

In [ ]:
# Initialize SIFT
sift = cv2.SIFT_create()
classes = 43  # Number of classes in the dataset

# Storage
data = []
labels = []

# Load images and extract SIFT features
for i in range(classes):
    path = os.path.join('gtsrb-german-traffic-sign/Train', str(i))
    images = os.listdir(path)
    
    for a in images:
        try:
            # Use OpenCV to load images quickly
            image_path = os.path.join(path, a)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale directly
            if image is None:
                continue
            
            image = cv2.resize(image, (10, 10))  # Resize for consistency
            
            # Extract SIFT features
            keypoints, descriptors = sift.detectAndCompute(image, None)
            if descriptors is not None:
                data.append(descriptors.flatten())  # Flatten for SVM
                labels.append(i)
        
        except Exception as e:
            print(f"Error loading image {a}: {e}")

# Convert lists to NumPy arrays
data = np.array(data, dtype='object')
labels = np.array(labels)

KeyboardInterrupt: 

In [ ]:
random.shuffle(data)

In [ ]:

print(len(data))

In [ ]:
from sklearn.cluster import KMeans

# Define number of visual words (clusters)
NUM_CLUSTERS = 100  # Lowering to 100 clusters to speed up training

# Convert variable-length SIFT descriptors into a single feature vector using K-Means
all_descriptors = np.vstack(data)  # Stack all extracted features
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init=10)
kmeans.fit(all_descriptors)

# Convert images to histograms of visual words
def compute_bovw_features(image_descriptors, kmeans_model, num_clusters=NUM_CLUSTERS):
    """Convert SIFT descriptors into Bag of Visual Words (BoVW) histograms."""
    histogram = np.zeros(num_clusters)
    if image_descriptors is not None:
        cluster_labels = kmeans_model.predict(image_descriptors)  # Assign features to clusters
        for label in cluster_labels:
            histogram[label] += 1
    return histogram

# Transform all training images into histograms
data_bovw = np.array([compute_bovw_features(desc, kmeans) for desc in data])

# Standardize features
scaler = StandardScaler()
data_bovw = scaler.fit_transform(data_bovw)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data_bovw, y_encoded, test_size=0.2, random_state=42)

# Train SVM
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# Save trained model
joblib.dump(svm_model, "svm_traffic_signs.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(kmeans, "kmeans_bovw.pkl")
joblib.dump(scaler, "scaler.pkl")

print("SVM Model trained and saved successfully!")

In [ ]:
def recognize_sign_svm(image, svm_model, kmeans_model, scaler, label_encoder):
    """Recognizes traffic signs using SIFT + BoVW + SVM classification."""
    sift = cv2.SIFT_create()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    if descriptors is None:
        return "Unknown"
    
    # Convert descriptors into a BoVW histogram
    bovw_features = compute_bovw_features(descriptors, kmeans_model).reshape(1, -1)
    bovw_features = scaler.transform(bovw_features)

    # Predict using SVM
    predicted_label = svm_model.predict(bovw_features)[0]

    # Convert back to sign name
    recognized_sign = label_encoder.inverse_transform([predicted_label])[0]
    return recognized_sign

In [ ]:
# Load trained models
svm_model = joblib.load("svm_traffic_signs.pkl")
label_encoder = joblib.load("label_encoder.pkl")
kmeans = joblib.load("kmeans_bovw.pkl")
scaler = joblib.load("scaler.pkl")

# Test on a new image
test_image_path = "GTSRB/Test/00000.png"
test_image = cv2.imread(test_image_path)
test_image = cv2.resize(test_image, (30,30))

recognized_sign = recognize_sign_svm(test_image, svm_model, kmeans, scaler, label_encoder)
print(f"Recognized Sign: {recognized_sign}")